In [1]:
import pandas as pd
import timeit

import sys
sys.path.insert(0, '../python')

from preprocessing import VCF, load_suffixes, prepare_input
from recalibrator import Recalibrator

In [2]:
trios = ["ajt", "chd", "corpas", "yri"]

# Pre-processing. Uncomment during first run of the script, then
# comment to avoid re-computing

# for trio in trios:
#     data_dir = '../data/' + trio + '/'
#     df = load_suffixes(data_dir)
#     df.to_csv(trio + '.csv', index=False)

Construct training dataset by concatenating rows from all the synthetic abortus trios

In [3]:
df_train = pd.DataFrame()
    
for train in trios:
    df_train = df_train.append(pd.read_csv(train + '.csv'))
    
# df_train = df_train[::1000] # Train on subset of input rows

In [4]:
X_train = prepare_input(df_train, target_cols=['justchild^GT'])
y_train = df_train['justchild^GT'].values

r = Recalibrator()


In [5]:
def callable(model):
    def inner():
        model.fit(X_train, y_train)
    
    return inner

models = [r.model_lr,
          r.model_xgb,
          r.model_meta]

model_names = ["Logistic Regression",
               "XGBoost",
               "Meta-classifier"]

for model, name in zip(models, model_names):
    print("{}: {} s".format(name, timeit.timeit(stmt=callable(model), 
                    number=20)))

Logistic Regression: 0.17235993999929633 s
XGBoost: 7.298937838000711 s
Meta-classifier: 6.97217417799402 s
